In [ ]:
# What is Resume Screening?
### Hiring the right talent is a challenge for all businesses. This challenge is magnified by the high volume of applicants if the business is labour-intensive, growing, and facing high attrition rates.
### An example of such a business is that IT departments are short of growing markets. In a typical service organization, professionals with a variety of technical skills and business domain expertise are hired and assigned to projects to resolve customer issues. This task of selecting the best talent among many others is known as Resume Screening.

### Typically, large companies do not have enough time to open each CV, so they use machine learning algorithms for the Resume Screening task.

In [ ]:
try:
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import warnings
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.multiclass import OneVsRestClassifier
    from sklearn import metrics
    from sklearn.metrics import accuracy_score
    from pandas.plotting import scatter_matrix
    from sklearn.neighbors import KNeighborsClassifier
except ModuleNotFoundError:
    !pip install -U numpy pandas matplotlib scikit-learn

In [ ]:
resumeDataSet = pd.read_csv('../input/resume-screening/Resume Screening.csv' ,encoding='utf-8')

In [ ]:
resumeDataSet['cleaned_resume'] = ''

In [ ]:
resumeDataSet.head()

In [ ]:
print ("Displaying the distinct categories of resume -")
print (resumeDataSet['Category'].unique())

In [ ]:
print ("Displaying the distinct categories of resume and the number of records belonging to each category -")
print (resumeDataSet['Category'].value_counts())

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=(15,15))
plt.xticks(rotation=90)
sns.countplot(y="Category",data=resumeDataSet)

In [ ]:
from matplotlib.gridspec import GridSpec

In [ ]:
targetCounts=resumeDataSet["Category"].value_counts()

In [ ]:
targetLabels=resumeDataSet["Category"].unique()

In [ ]:
plt.figure(1,figsize=(25,25))
the_grid=GridSpec(2,2)

cmap = plt.get_cmap('coolwarm')
colors = [cmap(i) for i in np.linspace(0, 1, 3)]
plt.subplot(the_grid[0, 1], aspect=1, title='CATEGORY DISTRIBUTION')

source_pie=plt.pie(targetCounts,labels=targetLabels,autopct='%1.1f%%', shadow=True, colors=colors)
plt.show()

In [ ]:
import re
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText

In [ ]:
resumeDataSet['cleaned_resume'] = resumeDataSet.Resume.apply(lambda x: cleanResume(x))

In [ ]:
import nltk
from nltk.corpus import stopwords
import string
from wordcloud import WordCloud

In [ ]:
oneSetOfStopWords = set(stopwords.words('english')+['``',"''"])

In [ ]:
totalWords =[]

In [ ]:
Sentences = resumeDataSet['Resume'].values

In [ ]:
cleanedSentences = ""

In [ ]:
for i in range(0,160):
    cleanedText = cleanResume(Sentences[i])
    cleanedSentences += cleanedText
    requiredWords = nltk.word_tokenize(cleanedText)
    for word in requiredWords:
        if word not in oneSetOfStopWords and word not in string.punctuation:
            totalWords.append(word)

In [ ]:
wordfreqdist = nltk.FreqDist(totalWords)

In [ ]:
mostcommon = wordfreqdist.most_common(50)

In [ ]:
mostcommon

In [ ]:
wc = WordCloud().generate(cleanedSentences)

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
var_mod = ['Category']
le = LabelEncoder()

In [ ]:
for i in var_mod:
    resumeDataSet[i] = le.fit_transform(resumeDataSet[i])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

In [ ]:
requiredText = resumeDataSet['cleaned_resume'].values

In [ ]:
requiredTarget = resumeDataSet['Category'].values

In [ ]:
requiredTarget = resumeDataSet['Category'].values

In [ ]:
WordFeatures = word_vectorizer.transform(requiredText)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(WordFeatures,requiredTarget,random_state=0, test_size=0.2)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
clf = OneVsRestClassifier(KNeighborsClassifier())

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
prediction = clf.predict(X_test)

In [ ]:
print('Accuracy of KNeighbors Classifier on training set: {:.2f}'.format(clf.score(X_train, y_train)))

In [ ]:
print('Accuracy of KNeighbors Classifier on test set: {:.2f}'.format(clf.score(X_test, y_test)))

In [ ]:
print("\n Classification report for classifier %s:\n%s\n" % (clf, metrics.classification_report(y_test, prediction)))